In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt

In [110]:
original = torch.rand((16, 3, 32, 32))

is_random_aug = (torch.rand((original.shape[0])) < 0.) * 1
is_ppo_aug = 1 - is_random_aug

random_aug_idx = torch.arange(0, len(is_random_aug))[is_random_aug.bool()]
ppo_aug_idx = torch.arange(0, len(is_ppo_aug))[is_ppo_aug.bool()]

print(random_aug_idx)
print(ppo_aug_idx)

random_imgs = original[random_aug_idx]
ppo_imgs = original[ppo_aug_idx]

tensor([], dtype=torch.int64)
tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])


In [113]:
random_imgs.shape[0]

0

In [108]:
return_img = torch.zeros_like(original)

return_img[random_aug_idx] = random_imgs
return_img[ppo_aug_idx] = ppo_imgs

In [116]:
dist = torch.distributions.Categorical(logits=torch.tensor([0.1, 0.4, 0.5]))

In [121]:
dist.sample(torch.tensor((100,)))

tensor([1, 1, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 0, 0, 0, 1, 2, 1, 1, 0, 2, 1, 1, 1,
        1, 1, 1, 2, 0, 0, 2, 2, 1, 0, 2, 2, 0, 2, 1, 0, 1, 0, 1, 1, 2, 2, 1, 0,
        1, 1, 0, 2, 0, 2, 1, 0, 0, 2, 0, 1, 2, 1, 1, 2, 1, 1, 1, 2, 2, 2, 1, 1,
        1, 1, 0, 0, 2, 2, 0, 2, 0, 1, 2, 2, 0, 2, 2, 0, 0, 0, 2, 2, 0, 0, 2, 1,
        0, 2, 2, 2])

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt

from utils.networks import DecoderNN_1input

In [7]:
decoder = DecoderNN_1input(
    num_transforms=5,
    num_discrete_magnitude=10,
    device='cpu'
)

In [9]:
color_magnitude_dist, color_permutation_dist = decoder.sample(50)

In [19]:
color_magnitude_index, color_permutation_index = color_magnitude_dist.sample((100,)), color_permutation_dist.sample((100,))

actions_index = torch.concat((
    torch.zeros((100, 2, 1), dtype=torch.float32),
    torch.zeros((100, 2, 1), dtype=torch.float32),
    color_magnitude_index.squeeze(1),
    color_permutation_index.squeeze(1).unsqueeze(-1),
    torch.zeros((100, 2, 1), dtype=torch.float32),
    torch.zeros((100, 2, 1), dtype=torch.float32),
    torch.zeros((100, 2, 1), dtype=torch.float32),
), dim=-1)

return actions_index

torch.Size([100, 2, 10])

In [ ]:
# [8, 2, 10] [10, 2, 10]
# (15, 2, 1) (15, 2, 1) (15, 2, 4) (15, 2, 1) (15, 2, 1) (15, 2, 1) (15, 2, 1)